In [67]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
import pandas as pd
import numpy as np
import datasurfer as ds
from pathlib import Path
from datasurfer.lib_llm.llm_agents import LLMAgent
from datasurfer.datautils import xml_valid_df

In [69]:
chapter = 3
version = 'R1V3'
root = Path(r'D:\02_Translation\01_Künstliche_Intelligenz_für_Ingenieure\03_KII_Chapter_03')

frcs  = root / f'KII_Chapter_{chapter:02}_Original.xlsx'
fdst1 = root / f'KII_Chapter_{chapter:02}_Translation_{version}.xlsx'
fdst2 = root / f'KII_Chapter_{chapter:02}_Translation_{version}.docx'
fbak  = root / f'KII_Chapter_{chapter:02}_Translation_{version}.csv'

In [70]:
Linda = LLMAgent('Linda', 'You are a Chinese linguist, you translate German to Chinese.')
Robin = LLMAgent('Robin', 'You are a Chinese linguist, you also know German.')

In [71]:
df_original = ds.AutoObject(frcs).df

df_original

,Page,Original,Translation,Review
0,69,Graphensuche,NaN,4
1,69,Dieses Kapitel behandelt die Grundidee von Suc...,NaN,4
2,69,3.1 Grundbegriffe der Graphentheorie,NaN,4
3,69,3.1.1 Vorgehensweise,NaN,4
4,69,Die Suche nach der Lösung eines Wissensverarbe...,NaN,4
...,...,...,...,...
660,115,Der Begriff des Zustands wird für die Darstell...,NaN,4
661,116,Literaturhinweise,NaN,4
662,116,Graphensuchalgorithmen sind in vielen Büchern ...,NaN,4
663,116,"Der Dijkstra-Algorithmus, der heute ein Standa...",NaN,4


In [72]:
pattern_Linda = 'Translating "{original}" to Chinese, return only the translation, do not include any other words.'
pattern_Robin = '根据德语原文\n"{original}"，\n将以下中文翻译改进到语义通顺, 修改其中的错误并去除不必要的句子：\n"{translation}"\n只返回修改过的句子.'
print(pattern_Robin)

根据德语原文
"{original}"，
将以下中文翻译改进到语义通顺, 修改其中的错误并去除不必要的句子：
"{translation}"
只返回修改过的句子.


In [73]:
def transview_text(original, memory_length=100):
    """
    Translates and reviews a given text.

    This function takes an original text and its translation, processes the translation
    using Linda's pattern, and then reviews the translation using Robin's pattern.

    Args:
        original (str): The original text to be translated and reviewed.
        translation (str): The initial translation of the original text.

    Returns:
        list: A list containing the original text and the reviewed translation.
    """
    while 1:

        translation = Linda.told(pattern_Linda.format(original=original), use_cache=False, memory_length=memory_length)

        if 'Instruction' not in translation:
            break

            
    reviewed = Robin.told(pattern_Robin.format(original=original, translation=translation), use_cache=True, memory_length=memory_length*2)
    return [original, reviewed] 

In [74]:

def start_translation(df, nrows):
    
    out = []
    buffer = []
    
    for idx, (page, original, trans, review) in df.iterrows():
        if review != 0:
            print(f'Processing {idx+1}/{nrows} ({(idx+1)/nrows*100:0.2f}%)...\n')
            Linda.print_message(f'{original}', 100, role='User')
        
        if review != 2:
            if len(buffer):
                merged = ' '.join(buffer)
                buffer = []
                out.append([page, *transview_text(merged)])     
                              
        if review == 4:
            out.append([page, *transview_text(original)])  
            
        elif review == 2:
            buffer.append(original)
            
        elif review == 1:
            continue
        
        elif review == 3:
            txts = original.split('@')
            
            for txt in txts: 
                out.append([page, *transview_text(txt)])
        
        elif review == 0:
            Linda.append_history(pattern_Linda.format(original=original), role='User')
            Linda.append_history(trans)
            Robin.append_history(pattern_Robin.format(original=original, translation=trans), role='User')
            Robin.append_history(trans)
            out.append([page, original, trans])
        else:
            raise ValueError(f'Invalid review value: {review}')
        
    return out

In [75]:
while 1:
    dfbak = pd.DataFrame() if not fbak.is_file() else pd.read_csv(fbak)
    idx_start = dfbak.index.max() + 1 if not dfbak.empty else 0
    
    if idx_start >= len(df_original):
        break
    
    df_working = dfbak.copy()
    df_working = pd.concat([df_working, df_original.loc[idx_start:min(idx_start+5, len(df_original)-1)]])
    df_working.fillna('', inplace=True)
       
    out = start_translation(df_working, len(df_original))
    
    pages, raw_text, translation = zip(*out)
    dfout = pd.DataFrame({'Page': pages, 'Original': raw_text, 'Translation': translation, 'Review': 0})
    dfout.Translation = dfout.Translation.str.replace('Let me know if you have more text to translate!', '')
    print('Saving...')
    dfbak = dfout #pd.concat([dfbak, dfout])
    dfbak.to_csv(fbak, index=False)


Processing 653/665 (98.20%)...

User:

Der Arbeitsspeicher (Datenbasis, data base) enthält die konkrete Aufgabenstellung, sowiedie während
der Lösung erzeugten Zwischenergebnisse, bei der Graphensuche also deneingelesenen Wurzelknoten
sowie die aktuellen Zwischenergebnisse der Suche, die u. a. inden Listen A, M und B gespeichert
sind.

----------------------------------------------------------------------------------------------------

Linda:

工作内存（数据基础）包含具体的任务设定以及在解决过程中产生的中间结果，对于图搜索来说，就是已读入的根节点以及搜索的当前中间结果，例如存储在列表 A、M 和 B
中的内容。   








--------------------------------------------------------------------------------

Robin:

工作内存（数据基础）包含具体的任务设定以及在解决过程中产生的中间结果，对于图搜索来说，就是已读入的根节点以及搜索的当前中间结果，例如存储在列表 A、M 和 B
中的内容。  




--------------------------------------------------------------------------------

Processing 654/665 (98.35%)...

User:

Als Operatoren werden diejenigen Vorschriften bezeichnet, mit denen der nächste Suchschritt
ausgeführt werden kann. Bei der Graphensuche bedeutet die An

In [76]:
dfbak.to_excel(fdst1, index=False)


In [77]:
obj = ds.DOCXObject(dfbak[['Original', 'Translation']], name='Translation')
obj.save_df(fdst2)

d:\13_dev\20_datasurfer\datasurfer\datainterface.py:38: UserWarning: Cannot import module "asammdf_object".
  warnings.warn(f'Cannot import module "{mdlname}".')
d:\13_dev\20_datasurfer\datasurfer\datainterface.py:38: UserWarning: Cannot import module "fin_object".
  warnings.warn(f'Cannot import module "{mdlname}".')
d:\13_dev\20_datasurfer\datasurfer\datainterface.py:38: UserWarning: Cannot import module "hdf_object".
  warnings.warn(f'Cannot import module "{mdlname}".')
d:\13_dev\20_datasurfer\datasurfer\datainterface.py:38: UserWarning: Cannot import module "matlab_object".
  warnings.warn(f'Cannot import module "{mdlname}".')
d:\13_dev\20_datasurfer\datasurfer\datainterface.py:38: UserWarning: Cannot import module "mdf_object".
  warnings.warn(f'Cannot import module "{mdlname}".')
d:\13_dev\20_datasurfer\datasurfer\datainterface.py:38: UserWarning: Cannot import module "parquet_object".
  warnings.warn(f'Cannot import module "{mdlname}".')
d:\13_dev\20_datasurfer\datasurfer\datain